In [ ]:
from gurobipy import Model, GRB, quicksum, LinExpr

In [ ]:
class Clustering_Model(object):
    def __init__(self, n_vertices, edges, constraints, k, gamma, 
                 verbosity=0, symmetry_breaking=1):
        self.check_graph(n_vertices, edges)
        self.n_vertices = n_vertices
        self.k = k
        self.verbosity = verbosity
        
        model = Model('graph_clustering')
        
        mvars = []
        for i in range(k):
            cvars = []
            for j in range(n_vertices):
                v = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
                cvars.append(v)
            mvars.append(cvars)
        model.update()
            
        # constraint: each vertex in exactly one cluster
        for v in range(n_vertices):
            model.addConstr(quicksum([mvars[i][v] for i in range(k)]), GRB.EQUAL, 1)
            
        # symmetry-breaking constraints
        if symmetry_breaking == 1:
            model.addConstr(mvars[0][0], GRB.EQUAL, 1)
            for i in range(2, k):
                model.addConstr(quicksum([mvars[i-1][j] for j in range(n_vertices)]) <=
                                quicksum([mvars[i][j] for j in range(n_vertices)]))
        
        
        obj_expr = LinExpr()
        
        # indicators for violation of cl constraints
        for (u, v, w) in constraints:
            for i in range(k):
                y = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
                model.update()
                model.addConstr(y >= mvars[i][u] + mvars[i][v] - 1)
                obj_expr.add(y, -w * gamma)
        
        # size of smallest cluster 
        s = model.addVar(lb=0.0, ub=n_vertices, vtype=GRB.INTEGER)
        model.update()
        for i in range(k):
            model.addConstr(s <= quicksum([mvars[i][v] for v in range(n_vertices)]))
        obj_expr.add(s)
        
        model.setObjective(obj_expr, GRB.MAXIMIZE)
        model.params.OutputFlag = self.verbosity
        model.Params.PreCrush = 1
        model.Params.LazyConstraints = 1
        
        model._cutfinder = Cut_Finder(n_vertices, edges)
        model._vars = mvars
        model._k = k
        model._relobj = None
        model._impcounter = 0
        
        self.model = model
               
    def check_graph(self, n_vertices, edges):
        vertices = set([i for (i, _) in edges])
        vertices |= set([i for (_, i) in edges])
        assert(vertices == set(range(n_vertices)))
        for u, v in edges:
            assert(u < v)
            assert(u < n_vertices)
    
    def solve(self):
        try:
            self.model.optimize(mincut_callback)
        except GurobiError:
            print(GurobiError.message)
        
        self.objective = None
        self.clusters = None
        self.optimal = (self.model.Status == GRB.OPTIMAL)
        self.runtime = self.model.Runtime
        
        if self.optimal:
            self.objective = self.model.ObjVal
            clusters = []
            for i in range(self.k):
                cluster = []
                for j in range(self.n_vertices):
                    if abs(self.model._vars[i][j].x) > 1e-4:
                        cluster.append(j)
                clusters.append(cluster)
            self.clusters = clusters
